# Recommendation Systems Movie


![family_movie](Images/family_movie.jpeg)


You pick the movie, I'll choose the restaurant...

## 1. Project Overview

This project aims to


### The Data

#### Source Data

This project uses the Movielens dataset from the [GroupLens](https://grouplens.org/datasets/movielens/latest/research) lab at the University of Minnesota, which can be found in in the `data` folder in this GitHub repository. 

The websits notes that "This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018. Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided."